In [ ]:
import json, time
from websocket import create_connection

In [15]:
import json
import websocket
import pandas as pd

def on_message(message):
    message = json.loads(message)

    if isinstance(message, dict):
        event = message.get("event")
        if event == "heartbeat":
            return  # Skip processing if the event is a heartbeat
        else:
            print(message)
    elif isinstance(message, list):
        
        print(message)
        # Extract the trade information from the message
        # Unpack the payload
        return
        id, data, channelName, pair = message

        # Extract the 'as' and 'bs' arrays
        asks = data.get('as')
        bids = data.get('bs')

        # Create empty lists to store the extracted data
        ask_prices, ask_volumes = [], []
        bid_prices, bid_volumes = [], []
        ids, pairs = [],[]

        ids.append(id)
        pairs.append(pairs)

        # Extract the data from the 'as' arrays
        for i, ask in enumerate(asks):
            ask_prices.append(ask[0])
            ask_volumes.append(ask[1])

        # Extract the data from the 'bs' arrays
        for i, bid in enumerate(bids):
            bid_prices.append(bid[0])
            bid_volumes.append(bid[1])

def on_error(ws, error):
    print(error)

def on_close(ws):
    print("Closed connection")

def on_open(ws):
    ws.send(json.dumps({
        "event": "subscribe",
        "pair": ["XBT/USD", "ETH/USD"],
        #"subscription": {"name": "trade"},
        #"subscription": {"name": "book", "depth": 10}
        "subscription": {"name": "ticker"}
    }))

if __name__ == "__main__":

    ws = create_connection("wss://ws.kraken.com")
    ws.send(json.dumps({
        "event": "subscribe",
        "pair": ["XBT/USD", "ETH/USD"],
        "subscription": {"name": "ticker"}
    }))
    while True:
        message = ws.recv()
        on_message(message)


--- request header ---
GET / HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: ws.kraken.com
Origin: http://ws.kraken.com
Sec-WebSocket-Key: rgvLBmonbZhoR9ILJbhetg==
Sec-WebSocket-Version: 13
Cookie: __cfruid=611b4a5b00f6557829ff297380d25cd94708c6da-1674495658; __cf_bm=wr00XyKmoIcjFRojvcd1bkd6KfD75zmlYhPhAkMtqLw-1674497363-0-AbWLVlbQcOEtbRZ+Hw7XvPbx4306k8U4U61qcDrxXSxrgIyn9/mAv8mjM0VoPGZY7W+aEWuEUcW/OYVyLJ/MxrE=


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Date: Mon, 23 Jan 2023 18:14:34 GMT
Connection: upgrade
Set-Cookie: AWSALB=nY+27ct5h2eo4VxJEFXYTGyfocU7WXSVgHe5FfcZ9K2nFhjnLtUsHfofAQyPJHKdxkgf/ZotJ704VookDT2zRniO3r/qgbm28WKFVNyLsxlvXulYE5Hvrtvc2vrG; Expires=Mon, 30 Jan 2023 18:14:34 GMT; Path=/
Set-Cookie: AWSALBCORS=nY+27ct5h2eo4VxJEFXYTGyfocU7WXSVgHe5FfcZ9K2nFhjnLtUsHfofAQyPJHKdxkgf/ZotJ704VookDT2zRniO3r/qgbm28WKFVNyLsxlvXulYE5Hvrtvc2vrG; Expires=Mon, 30 Jan 2023 18:14:34 GMT; Path=/; SameSite=None; Secure
Upgrade: websocket
Sec-WebSocket

{'connectionID': 17203464632379890586, 'event': 'systemStatus', 'status': 'online', 'version': '1.9.0'}
{'channelID': 340, 'channelName': 'ticker', 'event': 'subscriptionStatus', 'pair': 'XBT/USD', 'status': 'subscribed', 'subscription': {'name': 'ticker'}}
{'channelID': 564, 'channelName': 'ticker', 'event': 'subscriptionStatus', 'pair': 'ETH/USD', 'status': 'subscribed', 'subscription': {'name': 'ticker'}}
[340, {'a': ['23108.30000', 0, '0.10000000'], 'b': ['23104.60000', 0, '0.20000000'], 'c': ['23101.20000', '0.06740258'], 'v': ['2165.35103869', '3394.64345619'], 'p': ['22812.33863', '22730.44340'], 't': [18426, 27012], 'l': ['22520.10000', '22301.20000'], 'h': ['23127.10000', '23127.10000'], 'o': ['22717.10000', '22920.30000']}, 'ticker', 'XBT/USD']
[564, {'a': ['1630.87000', 3, '3.73922041'], 'b': ['1630.86000', 18, '18.43026125'], 'c': ['1631.07000', '0.14500000'], 'v': ['21300.61932641', '27671.02281705'], 'p': ['1624.96460', '1626.18482'], 't': [13199, 18047], 'l': ['1585.0000

KeyboardInterrupt: 

### ByteWax Dataflow

In [12]:
import json
from bytewax.inputs import ManualInputConfig
from bytewax.outputs import ManualOutputConfig

In [ ]:
PRODUCT_IDS = ["XBT/USD", "ETH/USD"]

In [ ]:
def ws_input(product_ids):
    state = None

    ws = create_connection("wss://ws.kraken.com")
    ws.send(json.dumps({
        "event": "subscribe",
        "pair": ["XBT/USD", "ETH/USD"],
        "subscription": {"name": "ticker"}
    }))

    print(ws.recv())
    while True:
        yield state, ws.recv()

In [ ]:
def input_builder(worker_index, worker_count, resume_state):
    # We're not going to worry about stateful recovery in this example.
    state = None
    prods_per_worker = int(len(PRODUCT_IDS) / worker_count)
    product_ids = PRODUCT_IDS[
        int(worker_index * prods_per_worker) : int(
            worker_index * prods_per_worker + prods_per_worker
        )
    ]
    return ws_input(product_ids, state)


input_config = ManualInputConfig(input_builder)

In [ ]:
def output_builder(worker_index, worker_count):
    return print

In [ ]:
from bytewax.dataflow import Dataflow


def key_on_product(data):
    return (data["product_id"], data)


flow = Dataflow()
flow.input("input", ManualInputConfig(input_builder))
flow.map(json.loads)
# {'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '36905.39', '0.00334873']], 'time': '2022-05-05T17:25:09.072519Z'}
flow.map(key_on_product)
# ('BTC-USD', {'type': 'l2update', 'product_id': 'BTC-USD', 'changes': [['buy', '36905.39', '0.00334873']], 'time': '2022-05-05T17:25:09.072519Z'})


In [ ]:
class OrderBook:
    def __init__(self):
        # if using Python < 3.7 need to use OrderedDict here
        self.bids = {}
        self.asks = {}

    def update(self, data):
        if self.bids == {}:
            self.bids = {float(price): float(size) for price, size in data["bids"]}
            # The bid_price is the highest priced buy limit order.
            # since the bids are in order, the first item of our newly constructed bids
            # will have our bid price, so we can track the best bid
            self.bid_price = next(iter(self.bids))
        if self.asks == {}:
            self.asks = {float(price): float(size) for price, size in data["asks"]}
            # The ask price is the lowest priced sell limit order.
            # since the asks are in order, the first item of our newly constructed
            # asks will be our ask price, so we can track the best ask
            self.ask_price = next(iter(self.asks))
        else:
            # We receive a list of lists here, normally it is only one change,
            # but could be more than one.
            for update in data["changes"]:
                price = float(update[1])
                size = float(update[2])
            if update[0] == "sell":
                # first check if the size is zero and needs to be removed
                if size == 0.0:
                    try:
                        del self.asks[price]
                        # if it was the ask price removed,
                        # update with new ask price
                        if price <= self.ask_price:
                            self.ask_price = min(self.asks.keys())
                    except KeyError:
                        # don't need to add price with size zero
                        pass
                else:
                    self.asks[price] = size
                    if price < self.ask_price:
                        self.ask_price = price
            if update[0] == "buy":
                # first check if the size is zero and needs to be removed
                if size == 0.0:
                    try:
                        del self.bids[price]
                        # if it was the bid price removed,
                        # update with new bid price
                        if price >= self.bid_price:
                            self.bid_price = max(self.bids.keys())
                    except KeyError:
                        # don't need to add price with size zero
                        pass
                else:
                    self.bids[price] = size
                    if price > self.bid_price:
                        self.bid_price = price
        return self, {
            "bid": self.bid_price,
            "bid_size": self.bids[self.bid_price],
            "ask": self.ask_price,
            "ask_price": self.asks[self.ask_price],
            "spread": self.ask_price - self.bid_price,
        }


flow.stateful_map("order_book", OrderBook, OrderBook.update)


In [ ]:
flow.filter(lambda x: x[-1]["spread"] / x[-1]["ask"] > 0.0001)
flow.capture(ManualOutputConfig(output_builder))